# Test Creation Notebook

In [1]:
from shutil import copyfile
import io, pytz, os, time, datetime
from os.path import dirname, join
import yaml
import pandas as pd
import numpy as np
import markdown

from IPython.display import display, Markdown, FileLink, FileLinks, clear_output, HTML
from IPython.core.display import HTML
from IPython.display import display, clear_output
from plotly.widgets import GraphWidget
    
import ipywidgets as widgets
from ipywidgets import interact

<IPython.core.display.Javascript object>

## Create Test

Input data below

In [2]:
frequency = '30S'
# frequency = '10Min'
yaml1 = {}

## Input generic data here
date = '2018-11-14'
test_date = pd.to_datetime(date)
# External or internal test
intORext = 'EXT'
# Name to append
append = 'SALEM_TEST'

## Global Test
yaml1['project'] = ('smartcitizen') # Input
yaml1['test'] = dict()
yaml1['test']['id'] = (test_date.strftime('%Y-%m') + '_' + intORext + '_' + append)
yaml1['commit_hash'] = ''

tz_widget = widgets.Dropdown(options=pytz.common_timezones, value='Europe/Madrid', description='Timezone: ')
display(tz_widget)

comment = '''---
**Comment:** 
Data from Salem for GIS course'''

yaml1['test']['author'] = 'Salem' # Input
yaml1['test']['comment'] = markdown.markdown(comment) # Input
yaml1['test']['type'] = ('outdoor') # Input
yaml1['test']['init_date'] = '2018-11-21' # Input
yaml1['test']['end_date'] = '2018-11-21' # Input
yaml1['test']['report'] = 'na' # Input

# Source
yaml1['test']['devices'] = dict()

## Add as many lines as desired
yaml1['test']['devices']['kits'] = dict()

## KIT 1
kit = 'Miltown'
yaml1['test']['devices']['kits'][kit] = dict()
yaml1['test']['devices']['kits'][kit]['SCK'] = 'KIT_1.5' # Input
yaml1['test']['devices']['kits'][kit]['PM'] = '' # Input
yaml1['test']['devices']['kits'][kit]['location'] = tz_widget.value
yaml1['test']['devices']['kits'][kit]['source'] = 'csv' # Input
yaml1['test']['devices']['kits'][kit]['type'] = 'KIT' # Input (STATION or KIT)
yaml1['test']['devices']['kits'][kit]['fileNameRaw'] = 'Miltown.csv' # Input
yaml1['test']['devices']['kits'][kit]['fileNameInfo'] = '' # Input
fileNameProcessed = (yaml1['test']['id'] + '_' + yaml1['test']['devices']['kits'][kit]['type'] + str(kit) + '.csv')
yaml1['test']['devices']['kits'][kit]['fileNameProc'] = fileNameProcessed
yaml1['test']['devices']['kits'][kit]['frequency'] = frequency # Min for minutes, S for seconds
#### Alphasense
# yaml1['test']['devices']['kits'][kit]['alphasense'] = dict()
# yaml1['test']['devices']['kits'][kit]['alphasense']['slots'] = ('CO', 'NO2', 'O3') # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['CO'] = 162581717 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['NO2'] = 202160410 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['O3'] = 204160162 # Input

## KIT 1
kit = 'UCD'
yaml1['test']['devices']['kits'][kit] = dict()
yaml1['test']['devices']['kits'][kit]['SCK'] = 'KIT_1.5' # Input
yaml1['test']['devices']['kits'][kit]['PM'] = '' # Input
yaml1['test']['devices']['kits'][kit]['location'] = tz_widget.value
yaml1['test']['devices']['kits'][kit]['source'] = 'csv' # Input
yaml1['test']['devices']['kits'][kit]['type'] = 'KIT' # Input (STATION or KIT)
yaml1['test']['devices']['kits'][kit]['fileNameRaw'] = 'UCD.csv' # Input
yaml1['test']['devices']['kits'][kit]['fileNameInfo'] = '' # Input
fileNameProcessed = (yaml1['test']['id'] + '_' + yaml1['test']['devices']['kits'][kit]['type'] + str(kit) + '.csv')
yaml1['test']['devices']['kits'][kit]['fileNameProc'] = fileNameProcessed
yaml1['test']['devices']['kits'][kit]['frequency'] = frequency # Min for minutes, S for seconds
#### Alphasense
# yaml1['test']['devices']['kits'][kit]['alphasense'] = dict()
# yaml1['test']['devices']['kits'][kit]['alphasense']['slots'] = ('CO', 'NO2', 'O3') # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['CO'] = 162581717 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['NO2'] = 202160410 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['O3'] = 204160162 # Input

## KIT 1
kit = 'Temple'
yaml1['test']['devices']['kits'][kit] = dict()
yaml1['test']['devices']['kits'][kit]['SCK'] = 'KIT_1.5' # Input
yaml1['test']['devices']['kits'][kit]['PM'] = '' # Input
yaml1['test']['devices']['kits'][kit]['location'] = tz_widget.value
yaml1['test']['devices']['kits'][kit]['source'] = 'csv' # Input
yaml1['test']['devices']['kits'][kit]['type'] = 'KIT' # Input (STATION or KIT)
yaml1['test']['devices']['kits'][kit]['fileNameRaw'] = 'Temple.csv' # Input
yaml1['test']['devices']['kits'][kit]['fileNameInfo'] = '' # Input
fileNameProcessed = (yaml1['test']['id'] + '_' + yaml1['test']['devices']['kits'][kit]['type'] + str(kit) + '.csv')
yaml1['test']['devices']['kits'][kit]['fileNameProc'] = fileNameProcessed
yaml1['test']['devices']['kits'][kit]['frequency'] = frequency # Min for minutes, S for seconds
#### Alphasense
# yaml1['test']['devices']['kits'][kit]['alphasense'] = dict()
# yaml1['test']['devices']['kits'][kit]['alphasense']['slots'] = ('CO', 'NO2', 'O3') # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['CO'] = 162581717 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['NO2'] = 202160410 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['O3'] = 204160162 # Input

## KIT 1
kit = 'Newcastle'
yaml1['test']['devices']['kits'][kit] = dict()
yaml1['test']['devices']['kits'][kit]['SCK'] = 'KIT_1.5' # Input
yaml1['test']['devices']['kits'][kit]['PM'] = '' # Input
yaml1['test']['devices']['kits'][kit]['location'] = tz_widget.value
yaml1['test']['devices']['kits'][kit]['source'] = 'csv' # Input
yaml1['test']['devices']['kits'][kit]['type'] = 'KIT' # Input (STATION or KIT)
yaml1['test']['devices']['kits'][kit]['fileNameRaw'] = 'Newcastle.csv' # Input
yaml1['test']['devices']['kits'][kit]['fileNameInfo'] = '' # Input
fileNameProcessed = (yaml1['test']['id'] + '_' + yaml1['test']['devices']['kits'][kit]['type'] + str(kit) + '.csv')
yaml1['test']['devices']['kits'][kit]['fileNameProc'] = fileNameProcessed
yaml1['test']['devices']['kits'][kit]['frequency'] = frequency # Min for minutes, S for seconds
#### Alphasense
# yaml1['test']['devices']['kits'][kit]['alphasense'] = dict()
# yaml1['test']['devices']['kits'][kit]['alphasense']['slots'] = ('CO', 'NO2', 'O3') # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['CO'] = 162581717 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['NO2'] = 202160410 # Input
# yaml1['test']['devices']['kits'][kit]['alphasense']['O3'] = 204160162 # Input

## Reference
yaml1['reference'] = dict()
yaml1['reference']['available'] = False # Input

if yaml1['reference']['available'] == True:
    yaml1['reference']['files'] = dict()
    
    ref_id_2 = 'ARPAE_MA'
    yaml1['reference']['files'][ref_id_2] = dict()
    yaml1['reference']['files'][ref_id_2]['location'] = tz_widget.value
    yaml1['reference']['files'][ref_id_2]['format'] = 'csv' # Input
    yaml1['reference']['files'][ref_id_2]['fileNameRaw'] = 'CaARPAE_MAData_2_COR.csv' # Input
    fileNameProcessed = (yaml1['test']['id'] + '_' + str(ref_id_1) + '_REF.csv')
    yaml1['reference']['files'][ref_id_2]['fileNameProc'] = fileNameProcessed # Input
    
    ## Reference timestamp name and format
    yaml1['reference']['files'][ref_id_2]['index'] = dict()
    yaml1['reference']['files'][ref_id_2]['index']['name'] = 'Time' # Input
    yaml1['reference']['files'][ref_id_2]['index']['format'] = '%Y-%m-%dT%H%M%S' # Input
    yaml1['reference']['files'][ref_id_2]['index']['frequency'] = '2S'# Min for minutes, S for seconds

    ## Reference channels and units
    yaml1['reference']['files'][ref_id_2]['channels'] = dict()
    yaml1['reference']['files'][ref_id_2]['channels']['pollutants'] = ('CO', 'O3', 'NO2', 'NO', 'NOX') # Input
    yaml1['reference']['files'][ref_id_2]['channels']['units'] = ('mg/m3', 'ug/m3', 'ug/m3', 'ug/m3', 'ug/m3') # Input
    yaml1['reference']['files'][ref_id_2]['channels']['names'] = ('CO mg/m3', 'O3 ug/m3', 'NO2 ug/m3', 'NO ug/m3', 'NOX ug/m3') # Inpu
    
# Create folder structure under data subdir
newpath = join(os.getcwd(), 'data', test_date.strftime('%Y'), test_date.strftime('%m'), yaml1['test']['id'])

update_ref = True

## Update Test

In [7]:
test_name = '2018-10_INT_TEST_TEMP_PM_CHARGE_SD'
test_year = test_name[0:4]
test_month = test_name[5:7]
newpath = join(os.getcwd(), 'data', test_year, test_month, test_name)
print newpath

with open(join(newpath, 'test_description.yaml'), 'r') as yaml_file:
    yaml1 = yaml.load(yaml_file)
    
display(yaml1)

/Users/macoscar/Documents/04_Projects/02_FabLab/02_SmartCitizen/04_iScape/99_DataAnalysis/smartcitizen-iscape-tests/data/2018/10/2018-10_INT_TEST_TEMP_PM_CHARGE_SD


{'commit_hash': '',
 'project': 'smartcitizen',
 'reference': {'available': False},
 'test': {'author': '\xc3\x93scar-Victor',
  'comment': u'<hr />\n<p><strong>Comment:</strong> \nTest for checking temperature of external and internal SHT31 with the PM and the Charger.\nDifferent recordings with PM ON, OFF, charging and not.</p>',
  'devices': {'kits': {'CHARGE_DISCHARGE_SD_NO_PM': {'PM': 'PMS5003',
     'SCK': 'KIT_2.1',
     'fileNameInfo': '',
     'fileNameProc': '2018-10_INT_TEST_TEMP_PM_CHARGE_KITCHARGE_DISCHARGE_SD_NO_PM.csv',
     'fileNameRaw': 'CHARGE_DISCHARGE_SD_NO_PM.csv',
     'frequency': '1S',
     'location': 'Europe/Madrid',
     'metadata': {'BATT': {'id': '10', 'unit': '%'},
      'BATT_CHG_RATE': {'id': '0', 'unit': 'mA'},
      'EXT_HUM': {'id': '80', 'unit': '%'},
      'EXT_TEMP': {'id': '79', 'unit': 'C'},
      'HUM': {'id': '56', 'unit': '%'},
      'LIGHT': {'id': '14', 'unit': 'Lux'},
      'NOISE_A': {'id': '53', 'unit': 'dBA'},
      'TEMP': {'id': '55',

In [10]:
## KIT 1
kit = 'CHARGE_DISCHARGE_CYCLE'
yaml1['test']['devices']['kits'][kit] = dict()
yaml1['test']['devices']['kits'][kit]['SCK'] = 'KIT_2.1' # Input
yaml1['test']['devices']['kits'][kit]['PM'] = 'PMS5003' # Input
yaml1['test']['devices']['kits'][kit]['location'] = 'Europe/Madrid'
yaml1['test']['devices']['kits'][kit]['source'] = 'csv' # Input
yaml1['test']['devices']['kits'][kit]['type'] = 'KIT' # Input (STATION or KIT)
yaml1['test']['devices']['kits'][kit]['fileNameRaw'] = '18-11-06.csv' # Input
yaml1['test']['devices']['kits'][kit]['fileNameInfo'] = '' # Input
fileNameProcessed = (yaml1['test']['id'] + '_' + yaml1['test']['devices']['kits'][kit]['type'] + str(kit) + '.csv')
yaml1['test']['devices']['kits'][kit]['fileNameProc'] = fileNameProcessed
yaml1['test']['devices']['kits'][kit]['frequency'] = '30S' # Min for minutes, S for seconds

update_ref = False

## Save Test

In [3]:
import csv

if not os.path.exists(newpath):
    os.makedirs(newpath)
    
# Copy RAW_DATA
raw_src_path = join(os.getcwd(), 'RAW_DATA')
raw_dst_path = join(newpath, 'RAW_DATA')
if not os.path.exists(raw_dst_path):
    os.makedirs(raw_dst_path)
for item in os.listdir(raw_src_path):
    if '.csv' in item or '.CSV' in item or '.TXT' in item :
        s = join(raw_src_path, item)
        d = join(raw_dst_path, item)
        copyfile(s, d)
        

## Check how many files we have
if yaml1['reference']['available'] == True and update_ref: 
    numFilesRef = len(yaml1['reference']['files'])
    
    for ref in yaml1['reference']['files']:
        src_path = join(os.getcwd(), 'RAW_DATA', yaml1['reference']['files'][ref]['fileNameRaw'])
        dst_path = join(newpath, (yaml1['test']['id'] + '_' + str(ref_id_1) + '_REF.csv'))
        timeName = yaml1['reference']['files'][ref]['index']['name']
        
        df = pd.read_csv(src_path, verbose=False, skiprows=[1]).set_index(timeName)
        df.index = pd.to_datetime(df.index) #.tz_localize('UTC').tz_convert(_tz_widget.value)
        df.sort_index(inplace=True)
        df = df.groupby(pd.TimeGrouper(freq=yaml1['reference']['files'][ref]['index']['frequency'])).aggregate(np.mean)
        df.drop([i for i in df.columns if 'Unnamed' in i], axis=1, inplace=True)
        
        df.to_csv(dst_path, sep=",")

numFilesKit = len(yaml1['test']['devices']['kits'])

for kit in yaml1['test']['devices']['kits']:
    src_path = join(os.getcwd(), 'RAW_DATA', yaml1['test']['devices']['kits'][kit]['fileNameRaw'])
    dst_path = join(newpath, (yaml1['test']['id'] + '_' + yaml1['test']['devices']['kits'][kit]['type'] + str(kit) + '.csv'))
    
    # Read file csv
    df = pd.read_csv(src_path, verbose=False, skiprows=range(1, 4)).set_index('Time')
    df.index = pd.to_datetime(df.index).tz_localize('UTC').tz_convert(tz_widget.value)
    df.sort_index(inplace=True)
    df.drop([i for i in df.columns if 'Unnamed' in i], axis=1, inplace=True)
        
    df.to_csv(dst_path, sep=",")
    
    ## Import units and ids
    dict_header = dict()
    with open(src_path, 'rb') as csvfile:
        readercsv = csv.reader(csvfile, delimiter = ',')
        line = 0
    
        header = next(readercsv)[1:]
        unit = next(readercsv)[1:]
        ids = next(readercsv)[1:]
    
        for key in header:
            dict_header[key] = dict()
            dict_header[key]['unit'] = unit[header.index(key)]
            dict_header[key]['id'] = ids[header.index(key)]
        
        yaml1['test']['devices']['kits'][kit]['metadata'] = dict_header
    
    ## Load txt info
    if yaml1['test']['devices']['kits'][kit]['fileNameInfo'] != '':
        src_path_info = join(os.getcwd(), 'RAW_DATA', yaml1['test']['devices']['kits'][kit]['fileNameInfo'])
        dict_info = dict()
        with open(src_path_info, 'rb') as infofile:
            for line in infofile:
                line = line.strip('\r\n')
                splitter = line.find(':')
                print line
                dict_info[line[:splitter]]= line[splitter+2:] # Accounting for the space
           
        yaml1['test']['devices']['kits'][kit]['info'] = dict_info
        
# Create yaml with test description
with open(join(newpath, 'test_description.yaml'), 'w') as yaml_file:
    yaml.dump(yaml1, yaml_file)
    
print 'Test Saved'

Test Saved


## Load Test

In [ ]:
readings = {}

def getTests(directory):
    tests = dict()
    mydir = join(directory, 'data')
    for root, dirs, files in os.walk(mydir):
        for _file in files:
            if _file.endswith(".yaml"):
                filePath = join(root, _file)
                stream = file(filePath)
                yamlFile = yaml.load(stream)
                tests[yamlFile['test']['id']] = root
                #print [yamlFile['test']['id'], filePath]
    return tests

def selectTests(x):
    global selectedTests
    selectedTests = list(x)

def loadTest(b):
    # print selectedTests
    
    for testPath in selectedTests:
        # Find Yaml
        filePath = join(testPath, 'test_description.yaml')
        stream = file(filePath)
        test = yaml.load(stream)
        test_id = test['test']['id']
        readings[test_id] = dict()
        
        # Open all kits
        for kit in test['test']['devices']['kits']:
            # Get fileName
            fileNameProc = test['test']['devices']['kits'][kit]['fileNameProc']
            fileData = join(testPath, fileNameProc)

            # Create pandas dataframe
            df = pd.read_csv(fileData, verbose=False, skiprows=[1]).set_index('TIME')
            # Create dictionary and add it to the readings key
            kitDict = dict()
            kitDict['data'] = df
            readings[test_id][kit] = kitDict
            
            ## Check if it's a STATION and retrieve alphadelta codes
            if test['test']['devices']['kits'][kit]['type'] == 'STATION':
                # print 'AlphaSense the sensor is'
                alphaDelta = dict()
                alphaDelta['CO'] = test['test']['devices']['kits'][kit]['alphasense']['CO']
                alphaDelta['NO2'] = test['test']['devices']['kits'][kit]['alphasense']['NO2']
                alphaDelta['O3'] = test['test']['devices']['kits'][kit]['alphasense']['O3']
                alphaDelta['SLOTS'] = test['test']['devices']['kits'][kit]['alphasense']['slots']
                readings[test_id][kit]['alphasense'] = alphaDelta
    
        ## Check if there's was a reference equipment during the test
        if test['reference']['available']:
            for reference in test['reference']['files']:
                # print 'Reference during the test was'
                referenceDict =  dict()
                # Get the file name
                fileNameProc = test['reference']['files'][reference]['fileNameProc']
                # Check the index name
                timeIndex = test['reference']['files'][reference]['index']['name']
                # Open it with pandas    
                fileData = join(testPath, fileNameProc)
                df = pd.read_csv(fileData, verbose=False, skiprows=[1]).set_index(timeIndex)
                    
                ## TODO: convert units here
                    
                referenceDict['data'] = df
                readings[test_id][reference] = referenceDict
                readings[test_id][reference]['is_reference'] = True
        

testsDir = os.getcwd()
tests = getTests(testsDir)

readings = {}
interact(selectTests,
         x = widgets.SelectMultiple(options=tests, 
                           description='Select tests below', 
                           selected_labels = selectedTests, 
                           layout=widgets.Layout(width='700px')))

loadB = widgets.Button(description='Load Tests')
loadB.on_click(loadTest)

display(loadB)